In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.dates as mdates
from matplotlib.colors import LinearSegmentedColormap, BoundaryNorm
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from shapely.geometry import Point, Polygon, box
from shapely.ops import unary_union, transform
import warnings
from datetime import datetime, timedelta
import os
import glob
from pathlib import Path
from scipy.interpolate import griddata
from scipy.ndimage import binary_dilation
import pyproj
from functools import partial
import seaborn as sns
from sklearn.metrics import brier_score_loss
from tqdm import tqdm
import pickle

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Set plotting style
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['legend.fontsize'] = 12
plt.rcParams['figure.dpi'] = 100

print("✅ All libraries imported successfully!")
print("📊 Matplotlib settings configured for validation plots")


In [ ]:
# Configuration and Paths
BASE_PATH = Path("/Users/jimnguyen/IRMII/SCS_API")
OUTLOOK_PATH = BASE_PATH / "convective_outlooks_only1200z"
PPH_HAIL_PATH = BASE_PATH / "PPH" / "NCEI_PPH" / "hail"
REPORTS_HAIL_PATH = BASE_PATH / "NCEI_storm_reports" / "hail_filtered"
NAM212_PATH = BASE_PATH / "PPH" / "nam212.nc"
CACHE_PATH = BASE_PATH / "cache"

# Output directory structure
OUTPUT_PATH = BASE_PATH / "validation_outputs"
FIGURES_PATH = OUTPUT_PATH / "figures"
BRIER_SCORES_PATH = FIGURES_PATH / "brier_scores"
VALIDATION_PATH = FIGURES_PATH / "validation"

START_YEAR = 2010
END_YEAR = 2024
HAIL_THRESHOLDS = [5, 15, 30]  # Probability thresholds to validate

# Create output directories
os.makedirs(CACHE_PATH, exist_ok=True)
os.makedirs(BRIER_SCORES_PATH, exist_ok=True)
os.makedirs(VALIDATION_PATH, exist_ok=True)

print(f"✅ Configuration set for {START_YEAR}-{END_YEAR}")
print(f"📁 Output directories created:")
print(f"   🎯 Brier Scores: {BRIER_SCORES_PATH}")
print(f"   ✅ Validation: {VALIDATION_PATH}")
print(f"   💾 Cache: {CACHE_PATH}")


In [ ]:
# Load NAM212 Grid
nam212_ds = xr.open_dataset(NAM212_PATH)
grid_lats = nam212_ds['gridlat_212'].values
grid_lons = nam212_ds['gridlon_212'].values
grid_shape = grid_lats.shape
CELL_AREA_KM2 = 40.6 * 40.6

print(f"🗺️ NAM212 Grid: {grid_shape[0]} x {grid_shape[1]} = {grid_shape[0] * grid_shape[1]:,} cells")
print(f"📏 Cell area: {CELL_AREA_KM2:,.1f} km² per cell")
print(f"🌍 Grid bounds: Lat [{np.min(grid_lats):.1f}, {np.max(grid_lats):.1f}], Lon [{np.min(grid_lons):.1f}, {np.max(grid_lons):.1f}]")


In [ ]:
# Utility Functions for Data Loading
def shapefile_to_nam212_grid(shapefile_path, threshold_value, grid_shape, grid_lons, grid_lats):
    """Convert shapefile polygons to NAM212 grid format with projection handling."""
    try:
        gdf = gpd.read_file(shapefile_path)
        grid = np.zeros(grid_shape)
        
        if len(gdf) == 0:
            return grid
        
        # Filter by threshold using DN column
        if 'DN' in gdf.columns:
            threshold_gdf = gdf[gdf['DN'] == threshold_value].copy()
        else:
            threshold_rows = []
            for idx, row in gdf.iterrows():
                if str(threshold_value) in str(row.to_dict()):
                    threshold_rows.append(row)
            if threshold_rows:
                threshold_gdf = gpd.GeoDataFrame(threshold_rows, crs=gdf.crs)
            else:
                return grid
        
        if len(threshold_gdf) == 0:
            return grid
        
        # Check projection and reproject if needed
        sample_geom = threshold_gdf.iloc[0].geometry
        if sample_geom is not None:
            if hasattr(sample_geom, 'exterior'):
                x, y = list(sample_geom.exterior.coords)[0]
            else:
                x, y = list(sample_geom.geoms[0].exterior.coords)[0]
            
            # If coordinates > 180, it's projected (Lambert Conformal Conic)
            if abs(x) > 180 or abs(y) > 90:
                threshold_gdf = threshold_gdf.to_crs('EPSG:4326')
        
        # Merge polygons and check grid points
        all_geoms = [row.geometry for idx, row in threshold_gdf.iterrows() 
                    if row.geometry is not None and row.geometry.is_valid]
        
        if not all_geoms:
            return grid
            
        merged_geom = unary_union(all_geoms)
        if merged_geom.is_empty:
            return grid
            
        minx, miny, maxx, maxy = merged_geom.bounds
        
        for i in range(grid_shape[0]):
            for j in range(grid_shape[1]):
                lon, lat = grid_lons[i, j], grid_lats[i, j]
                if minx <= lon <= maxx and miny <= lat <= maxy:
                    if merged_geom.contains(Point(lon, lat)):
                        grid[i, j] = 1.0
        
        return grid
        
    except Exception as e:
        print(f"   ⚠️ Error processing {shapefile_path}: {e}")
        return np.zeros(grid_shape)

def get_outlook_path(date, hazard_type='hail'):
    """Get outlook shapefile path."""
    year, month, day = date.year, date.month, date.day
    base_path = OUTLOOK_PATH / str(year) / str(month) / "forecast_day1"
    outlook_dir = base_path / f"day1otlk_{date.strftime('%Y%m%d')}_1200"
    
    if hazard_type == 'hail':
        shapefile = outlook_dir / f"day1otlk_{date.strftime('%Y%m%d')}_1200_hail.shp"
    else:
        shapefile = outlook_dir / f"day1otlk_{date.strftime('%Y%m%d')}_1200_sighail.shp"
    
    return shapefile if shapefile.exists() else None

def load_pph_data(date, hazard_type='hail'):
    """Load PPH data for a specific date."""
    if hazard_type == 'hail':
        pph_file = PPH_HAIL_PATH / f"pph_{date.strftime('%Y_%m_%d')}.csv"
    else:
        pph_file = PPH_SIGHAIL_PATH / f"pph_{date.strftime('%Y_%m_%d')}.csv"
    
    if pph_file.exists():
        return pd.read_csv(pph_file, header=0).values
    else:
        return None

def load_storm_reports(date):
    """Load hail storm reports for a specific date."""
    report_file = REPORTS_HAIL_PATH / f"hail_reports_{date.strftime('%Y_%m_%d')}.csv"
    
    if report_file.exists():
        return pd.read_csv(report_file)
    else:
        return None

def storm_reports_to_grid(reports_df, grid_shape, grid_lons, grid_lats, buffer_km=40.6):
    """Convert storm reports to binary grid with optional buffering."""
    grid = np.zeros(grid_shape)
    
    if reports_df is None or len(reports_df) == 0:
        return grid
    
    # Convert buffer from km to degrees (rough approximation)
    buffer_deg = buffer_km / 111.0  # ~111 km per degree
    
    for _, report in reports_df.iterrows():
        report_lat = report['BEGIN_LAT']
        report_lon = report['BEGIN_LON']
        
        # Find nearest grid points within buffer
        lat_diff = np.abs(grid_lats - report_lat)
        lon_diff = np.abs(grid_lons - report_lon)
        distance = np.sqrt(lat_diff**2 + lon_diff**2)
        
        # Set grid points within buffer to 1
        grid[distance <= buffer_deg] = 1.0
    
    return grid

print("✅ Utility functions defined")


In [ ]:
# Brier Score and Validation Functions
def calculate_brier_score(forecasts, observations):
    """
    Calculate Brier Score for probabilistic forecasts.
    
    BS = (1/N) * Σ(forecast_prob - observation)²
    
    Args:
        forecasts: Array of forecast probabilities (0-1)
        observations: Array of binary observations (0 or 1)
    
    Returns:
        Brier Score (lower is better, perfect score = 0)
    """
    forecasts = np.asarray(forecasts).flatten()
    observations = np.asarray(observations).flatten()
    
    # Remove NaN values
    valid_mask = ~(np.isnan(forecasts) | np.isnan(observations))
    forecasts = forecasts[valid_mask]
    observations = observations[valid_mask]
    
    if len(forecasts) == 0:
        return np.nan
    
    return np.mean((forecasts - observations) ** 2)

def calculate_brier_skill_score(forecast_bs, reference_bs):
    """
    Calculate Brier Skill Score relative to reference forecast.
    
    BSS = 1 - (BS_forecast / BS_reference)
    
    Positive values indicate skill above reference.
    """
    if reference_bs == 0 or np.isnan(reference_bs) or np.isnan(forecast_bs):
        return np.nan
    
    return 1 - (forecast_bs / reference_bs)

def calculate_climatology(observations):
    """
    Calculate climatological forecast (base rate).
    """
    observations = np.asarray(observations).flatten()
    valid_obs = observations[~np.isnan(observations)]
    
    if len(valid_obs) == 0:
        return np.nan
    
    return np.mean(valid_obs)

def process_validation_data(threshold, start_year=START_YEAR, end_year=END_YEAR):
    """
    Process validation data for a specific threshold across multiple years.
    
    Returns:
        Dictionary with PPH forecasts, outlook forecasts, and observations
    """
    cache_file = CACHE_PATH / f"validation_data_hail_{threshold}pct_{start_year}_{end_year}.pkl"
    
    if cache_file.exists():
        print(f"   📦 Loading cached validation data for {threshold}%")
        with open(cache_file, 'rb') as f:
            return pickle.load(f)
    
    print(f"   🔄 Processing validation data for {threshold}%")
    
    pph_forecasts = []
    outlook_forecasts = []
    observations = []
    valid_dates = []
    
    start_date = datetime(start_year, 1, 1)
    end_date = datetime(end_year, 12, 31)
    current_date = start_date
    
    processed_days = 0
    total_days = (end_date - start_date).days + 1
    
    while current_date <= end_date:
        # Load PPH data
        pph_data = load_pph_data(current_date, 'hail')
        
        # Load Outlook data (issued previous day)
        outlook_date = current_date - timedelta(days=1)
        outlook_path = get_outlook_path(outlook_date, 'hail')
        outlook_grid = None
        if outlook_path and outlook_path.exists():
            try:
                outlook_grid = shapefile_to_nam212_grid(outlook_path, threshold, grid_shape, grid_lons, grid_lats)
            except:
                outlook_grid = None
        
        # Load storm reports
        reports = load_storm_reports(current_date)
        obs_grid = storm_reports_to_grid(reports, grid_shape, grid_lons, grid_lats)
        
        # Only include days where we have all three datasets
        if pph_data is not None and outlook_grid is not None:
            # Convert PPH to probability grid
            pph_prob = pph_data / 100.0  # Convert percentage to probability
            
            pph_forecasts.append(pph_prob.flatten())
            outlook_forecasts.append(outlook_grid.flatten())
            observations.append(obs_grid.flatten())
            valid_dates.append(current_date)
            processed_days += 1
        
        current_date += timedelta(days=1)
        
        # Progress update
        if processed_days % 100 == 0:
            progress = ((current_date - start_date).days / total_days) * 100
            print(f"      📊 Progress: {progress:.1f}% ({processed_days} valid days)")
    
    print(f"      ✓ Processed {processed_days} days with complete data")
    
    # Stack all data
    validation_data = {
        'pph_forecasts': np.vstack(pph_forecasts) if pph_forecasts else np.array([]),
        'outlook_forecasts': np.vstack(outlook_forecasts) if outlook_forecasts else np.array([]),
        'observations': np.vstack(observations) if observations else np.array([]),
        'dates': valid_dates,
        'threshold': threshold,
        'processed_days': processed_days
    }
    
    # Cache results
    with open(cache_file, 'wb') as f:
        pickle.dump(validation_data, f)
    
    return validation_data

print("✅ Brier score and validation functions defined")


In [ ]:
# Validation Analysis Functions
def compute_comprehensive_validation(validation_data):
    """
    Compute comprehensive validation metrics for PPH and Outlook forecasts.
    """
    threshold = validation_data['threshold']
    pph_forecasts = validation_data['pph_forecasts']
    outlook_forecasts = validation_data['outlook_forecasts']
    observations = validation_data['observations']
    
    print(f"\n📊 Computing validation metrics for {threshold}% threshold")
    print(f"   📈 Data shape: {pph_forecasts.shape}")
    print(f"   📅 Days: {len(validation_data['dates'])}")
    
    # Flatten all arrays for computation
    pph_flat = pph_forecasts.flatten()
    outlook_flat = outlook_forecasts.flatten()
    obs_flat = observations.flatten()
    
    # Calculate climatology (base rate)
    climatology = calculate_climatology(obs_flat)
    climatology_forecast = np.full_like(obs_flat, climatology)
    
    # Calculate Brier Scores
    pph_bs = calculate_brier_score(pph_flat, obs_flat)
    outlook_bs = calculate_brier_score(outlook_flat, obs_flat)
    climatology_bs = calculate_brier_score(climatology_forecast, obs_flat)
    
    # Calculate Brier Skill Scores
    pph_bss = calculate_brier_skill_score(pph_bs, climatology_bs)
    outlook_bss = calculate_brier_skill_score(outlook_bs, climatology_bs)
    
    # Additional metrics
    event_frequency = np.mean(obs_flat)
    pph_mean_forecast = np.mean(pph_flat)
    outlook_mean_forecast = np.mean(outlook_flat)
    
    results = {
        'threshold': threshold,
        'climatology': climatology,
        'event_frequency': event_frequency,
        'pph_brier_score': pph_bs,
        'outlook_brier_score': outlook_bs,
        'climatology_brier_score': climatology_bs,
        'pph_brier_skill_score': pph_bss,
        'outlook_brier_skill_score': outlook_bss,
        'pph_mean_forecast': pph_mean_forecast,
        'outlook_mean_forecast': outlook_mean_forecast,
        'total_grid_points': len(obs_flat),
        'total_events': np.sum(obs_flat)
    }
    
    # Print summary
    print(f"\n📋 VALIDATION SUMMARY - {threshold}% Threshold")
    print(f"   🎯 Event Frequency: {event_frequency:.4f} ({event_frequency*100:.2f}%)")
    print(f"   📊 Total Events: {int(np.sum(obs_flat)):,} / {len(obs_flat):,} grid points")
    print(f"\n🎯 BRIER SCORES (lower is better):")
    print(f"   🔮 PPH: {pph_bs:.6f}")
    print(f"   📡 Outlook: {outlook_bs:.6f}")
    print(f"   🌡️ Climatology: {climatology_bs:.6f}")
    print(f"\n⭐ BRIER SKILL SCORES (higher is better):")
    print(f"   🔮 PPH BSS: {pph_bss:.4f}")
    print(f"   📡 Outlook BSS: {outlook_bss:.4f}")
    print(f"\n📈 MEAN FORECASTS:")
    print(f"   🔮 PPH: {pph_mean_forecast:.4f} ({pph_mean_forecast*100:.2f}%)")
    print(f"   📡 Outlook: {outlook_mean_forecast:.4f} ({outlook_mean_forecast*100:.2f}%)")
    
    return results

def plot_validation_summary(all_results):
    """
    Create comprehensive validation summary plots.
    """
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
    
    thresholds = [r['threshold'] for r in all_results]
    pph_bs = [r['pph_brier_score'] for r in all_results]
    outlook_bs = [r['outlook_brier_score'] for r in all_results]
    climatology_bs = [r['climatology_brier_score'] for r in all_results]
    pph_bss = [r['pph_brier_skill_score'] for r in all_results]
    outlook_bss = [r['outlook_brier_skill_score'] for r in all_results]
    
    # Plot 1: Brier Scores
    x_pos = np.arange(len(thresholds))
    width = 0.25
    
    ax1.bar(x_pos - width, pph_bs, width, label='PPH', color='#2E8B57', alpha=0.8)
    ax1.bar(x_pos, outlook_bs, width, label='Outlook', color='#FF6B6B', alpha=0.8)
    ax1.bar(x_pos + width, climatology_bs, width, label='Climatology', color='#95A5A6', alpha=0.8)
    
    ax1.set_xlabel('Threshold (%)', fontweight='bold')
    ax1.set_ylabel('Brier Score', fontweight='bold')
    ax1.set_title('Brier Scores by Threshold\n(Lower is Better)', fontweight='bold')
    ax1.set_xticks(x_pos)
    ax1.set_xticklabels([f'{t}%' for t in thresholds])
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Brier Skill Scores
    ax2.bar(x_pos - width/2, pph_bss, width, label='PPH', color='#2E8B57', alpha=0.8)
    ax2.bar(x_pos + width/2, outlook_bss, width, label='Outlook', color='#FF6B6B', alpha=0.8)
    ax2.axhline(y=0, color='black', linestyle='--', alpha=0.5)
    
    ax2.set_xlabel('Threshold (%)', fontweight='bold')
    ax2.set_ylabel('Brier Skill Score', fontweight='bold')
    ax2.set_title('Brier Skill Scores by Threshold\n(Higher is Better)', fontweight='bold')
    ax2.set_xticks(x_pos)
    ax2.set_xticklabels([f'{t}%' for t in thresholds])
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Plot 3: Event Frequencies
    event_freq = [r['event_frequency']*100 for r in all_results]
    
    ax3.bar(x_pos, event_freq, color='#3498DB', alpha=0.8)
    ax3.set_xlabel('Threshold (%)', fontweight='bold')
    ax3.set_ylabel('Event Frequency (%)', fontweight='bold')
    ax3.set_title('Observed Event Frequency by Threshold', fontweight='bold')
    ax3.set_xticks(x_pos)
    ax3.set_xticklabels([f'{t}%' for t in thresholds])
    ax3.grid(True, alpha=0.3)
    
    # Plot 4: Mean Forecast Values
    pph_mean = [r['pph_mean_forecast']*100 for r in all_results]
    outlook_mean = [r['outlook_mean_forecast']*100 for r in all_results]
    
    ax4.bar(x_pos - width/2, pph_mean, width, label='PPH Mean', color='#2E8B57', alpha=0.8)
    ax4.bar(x_pos + width/2, outlook_mean, width, label='Outlook Mean', color='#FF6B6B', alpha=0.8)
    ax4.plot(x_pos, event_freq, 'ko-', linewidth=2, markersize=8, label='Observed Rate')
    
    ax4.set_xlabel('Threshold (%)', fontweight='bold')
    ax4.set_ylabel('Forecast Rate (%)', fontweight='bold')
    ax4.set_title('Mean Forecast vs Observed Rates', fontweight='bold')
    ax4.set_xticks(x_pos)
    ax4.set_xticklabels([f'{t}%' for t in thresholds])
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    # Save plot
    save_path = VALIDATION_PATH / f"brier_score_validation_summary_{START_YEAR}-{END_YEAR}.png"
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    print(f"\n💾 Saved validation summary to {save_path}")
    
    plt.show()
    return fig

print("✅ Validation analysis functions defined")


In [ ]:
# TASK 1: Process Validation Data for All Thresholds
print("🔄 PROCESSING VALIDATION DATA FOR ALL THRESHOLDS")
print("="*60)

validation_datasets = {}

for threshold in HAIL_THRESHOLDS:
    print(f"\n📊 Processing {threshold}% threshold...")
    validation_data = process_validation_data(threshold)
    validation_datasets[threshold] = validation_data
    
    print(f"   ✅ {threshold}% threshold complete")
    print(f"   📈 Shape: {validation_data['pph_forecasts'].shape}")
    print(f"   📅 Valid days: {validation_data['processed_days']}")

print("\n✅ All validation data processed and cached")


In [ ]:
# TASK 2: Compute Brier Scores and Validation Metrics
print("🎯 COMPUTING BRIER SCORES AND VALIDATION METRICS")
print("="*60)

all_validation_results = []

for threshold in HAIL_THRESHOLDS:
    print(f"\n🔄 Computing metrics for {threshold}% threshold...")
    validation_data = validation_datasets[threshold]
    results = compute_comprehensive_validation(validation_data)
    all_validation_results.append(results)

print("\n✅ All validation metrics computed")


In [ ]:
# TASK 3: Create Validation Summary Plots
print("📊 CREATING VALIDATION SUMMARY PLOTS")
print("="*50)

validation_fig = plot_validation_summary(all_validation_results)

print("\n✅ Validation summary plots completed")


In [ ]:
# TASK 4: Create Detailed Results Table
print("📋 CREATING DETAILED RESULTS TABLE")
print("="*50)

# Create comprehensive results table
results_data = []
for result in all_validation_results:
    results_data.append({
        'Threshold (%)': result['threshold'],
        'Event Frequency (%)': f"{result['event_frequency']*100:.3f}",
        'Total Events': f"{int(result['total_events']):,}",
        'PPH Brier Score': f"{result['pph_brier_score']:.6f}",
        'Outlook Brier Score': f"{result['outlook_brier_score']:.6f}",
        'Climatology Brier Score': f"{result['climatology_brier_score']:.6f}",
        'PPH Brier Skill Score': f"{result['pph_brier_skill_score']:.4f}",
        'Outlook Brier Skill Score': f"{result['outlook_brier_skill_score']:.4f}",
        'PPH Mean Forecast (%)': f"{result['pph_mean_forecast']*100:.3f}",
        'Outlook Mean Forecast (%)': f"{result['outlook_mean_forecast']*100:.3f}"
    })

results_df = pd.DataFrame(results_data)

print("\n📊 COMPREHENSIVE VALIDATION RESULTS")
print("="*80)
print(results_df.to_string(index=False))

# Save results table
results_path = VALIDATION_PATH / f"brier_score_results_{START_YEAR}-{END_YEAR}.csv"
results_df.to_csv(results_path, index=False)
print(f"\n💾 Saved detailed results to {results_path}")

print("\n✅ Detailed results table completed")


In [ ]:
# TASK 5: Performance Interpretation and Insights
print("🔍 PERFORMANCE INTERPRETATION AND INSIGHTS")
print("="*60)

print("\n📈 KEY FINDINGS:")
print("="*40)

for i, result in enumerate(all_validation_results):
    threshold = result['threshold']
    pph_bss = result['pph_brier_skill_score']
    outlook_bss = result['outlook_brier_skill_score']
    
    print(f"\n🎯 {threshold}% Threshold:")
    
    # Determine better performing model
    if pph_bss > outlook_bss:
        better = "PPH"
        advantage = pph_bss - outlook_bss
        print(f"   🏆 PPH outperforms Outlook (BSS advantage: +{advantage:.4f})")
    elif outlook_bss > pph_bss:
        better = "Outlook"
        advantage = outlook_bss - pph_bss
        print(f"   🏆 Outlook outperforms PPH (BSS advantage: +{advantage:.4f})")
    else:
        print(f"   🤝 PPH and Outlook perform equally")
    
    # Skill assessment
    if pph_bss > 0:
        print(f"   ✅ PPH shows positive skill vs climatology")
    else:
        print(f"   ❌ PPH shows no skill vs climatology")
        
    if outlook_bss > 0:
        print(f"   ✅ Outlook shows positive skill vs climatology")
    else:
        print(f"   ❌ Outlook shows no skill vs climatology")
    
    # Event frequency insights
    event_freq = result['event_frequency'] * 100
    if event_freq < 1:
        print(f"   📊 Rare events ({event_freq:.3f}% frequency) - challenging to forecast")
    elif event_freq < 5:
        print(f"   📊 Uncommon events ({event_freq:.3f}% frequency) - moderate difficulty")
    else:
        print(f"   📊 Common events ({event_freq:.3f}% frequency) - easier to forecast")

# Overall comparison
pph_avg_bss = np.mean([r['pph_brier_skill_score'] for r in all_validation_results])
outlook_avg_bss = np.mean([r['outlook_brier_skill_score'] for r in all_validation_results])

print(f"\n🎯 OVERALL PERFORMANCE:")
print(f"   📊 PPH Average BSS: {pph_avg_bss:.4f}")
print(f"   📊 Outlook Average BSS: {outlook_avg_bss:.4f}")

if pph_avg_bss > outlook_avg_bss:
    print(f"   🏆 PPH shows superior overall performance")
elif outlook_avg_bss > pph_avg_bss:
    print(f"   🏆 Outlook shows superior overall performance")
else:
    print(f"   🤝 PPH and Outlook show equivalent overall performance")

print("\n✅ Performance interpretation completed")
